In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow import keras

In [2]:
df = pd.read_csv("modified_file.csv")

In [3]:
df

,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Type
0,0.170,0.29,1.4,0.047,23.0,107.0,0.99390,3.52,0.65,10.4,6,White Wine
1,0.310,0.38,10.5,0.031,53.0,140.0,0.99321,3.34,0.46,11.7,6,White Wine
2,0.145,0.29,1.0,0.042,35.0,90.0,0.99080,3.76,0.49,11.3,6,White Wine
3,0.260,0.29,4.2,0.043,33.0,114.0,0.99020,3.16,0.31,12.5,6,White Wine
4,0.450,0.07,1.1,0.030,10.0,131.0,0.99050,2.97,0.28,10.8,5,White Wine
...,...,...,...,...,...,...,...,...,...,...,...,...
6444,0.190,0.31,14.5,0.045,39.0,193.0,0.99860,3.10,0.50,9.2,6,White Wine
6445,0.160,0.42,1.0,0.036,29.0,113.0,0.99080,3.18,0.52,11.0,6,White Wine
6446,0.250,0.26,13.1,0.051,44.0,148.0,0.99720,3.29,0.38,9.3,5,White Wine
6447,0.250,0.46,1.7,0.062,26.0,42.0,0.99590,3.18,0.83,10.6,6,Red Wine


In [4]:
characteristics = [
    'volatile acidity', 'citric acid', 'residual sugar', 'chlorides',
    'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH',
    'sulphates', 'alcohol'
]

# Iterate through each characteristic and find the smallest and largest values
for characteristic in characteristics:
    min_value = df[characteristic].min()
    max_value = df[characteristic].max()
    print(f"{characteristic}: Smallest = {min_value}, Largest = {max_value}")

volatile acidity: Smallest = 0.08, Largest = 1.58
citric acid: Smallest = 0.0, Largest = 1.66
residual sugar: Smallest = 0.6, Largest = 65.8
chlorides: Smallest = 0.009, Largest = 0.611
free sulfur dioxide: Smallest = 1.0, Largest = 289.0
total sulfur dioxide: Smallest = 6.0, Largest = 440.0
density: Smallest = 0.98711, Largest = 1.03898
pH: Smallest = 2.72, Largest = 4.01
sulphates: Smallest = 0.22, Largest = 2.0
alcohol: Smallest = 8.0, Largest = 14.9


In [5]:
# Perform one-hot encoding for 'Type' column
df = pd.get_dummies(df, columns=['Type'], drop_first=True)


In [6]:
# Split data into features (X) and target (y)
X = df.drop(columns=['quality'])
y = df['quality']


In [7]:
print(X)
print(y)

      volatile acidity  citric acid  residual sugar  chlorides   
0                0.170         0.29             1.4      0.047  \
1                0.310         0.38            10.5      0.031   
2                0.145         0.29             1.0      0.042   
3                0.260         0.29             4.2      0.043   
4                0.450         0.07             1.1      0.030   
...                ...          ...             ...        ...   
6444             0.190         0.31            14.5      0.045   
6445             0.160         0.42             1.0      0.036   
6446             0.250         0.26            13.1      0.051   
6447             0.250         0.46             1.7      0.062   
6448             0.540         0.21            16.3      0.055   

      free sulfur dioxide  total sulfur dioxide  density    pH  sulphates   
0                    23.0                 107.0  0.99390  3.52       0.65  \
1                    53.0                 140.0  0.99

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



# Standardize features (optional but recommended for neural networks)
scaler = StandardScaler(copy=False)
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),  # Input layer
    keras.layers.Dense(128, activation='relu'),     
    keras.layers.Dropout(0.2),                     
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)
])

In [9]:
from keras.optimizers import Adam
model.compile(
    optimizer=Adam(learning_rate=0.001),  # Use the Adam optimizer
    loss='mean_squared_error',  # Specify the loss function
    metrics=['mse'],  # Optional: specify metrics to monitor during training
)

In [10]:
# Train the model
history = model.fit(
    X_train, y_train, 
    epochs=500, 
    batch_size=64, 
    validation_split=0.2
)

Epoch 1/500
65/65 [==============================] - 3s 17ms/step - loss: 9.7391 - mse: 9.7391 - val_loss: 1.8633 - val_mse: 1.8633
Epoch 2/500
65/65 [==============================] - 1s 9ms/step - loss: 2.2659 - mse: 2.2659 - val_loss: 1.1956 - val_mse: 1.1956
Epoch 3/500
65/65 [==============================] - 1s 8ms/step - loss: 1.6972 - mse: 1.6972 - val_loss: 1.0101 - val_mse: 1.0101
Epoch 4/500
65/65 [==============================] - 1s 8ms/step - loss: 1.4173 - mse: 1.4173 - val_loss: 0.8501 - val_mse: 0.8501
Epoch 5/500
65/65 [==============================] - 1s 8ms/step - loss: 1.2292 - mse: 1.2292 - val_loss: 0.6807 - val_mse: 0.6807
Epoch 6/500
65/65 [==============================] - 1s 9ms/step - loss: 1.0917 - mse: 1.0917 - val_loss: 0.6331 - val_mse: 0.6331
Epoch 7/500
65/65 [==============================] - 1s 10ms/step - loss: 1.0274 - mse: 1.0274 - val_loss: 0.6258 - val_mse: 0.6258
Epoch 8/500
65/65 [==============================] - 1s 10ms/step - loss: 0.9708 

In [11]:
y_pred = model.predict(X_test)
print(y_pred)

41/41 [==============================] - 0s 2ms/step
[[5.1243052]
 [6.6905146]
 [5.672873 ]
 ...
 [6.311339 ]
 [6.110047 ]
 [5.2496176]]


In [12]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean squared error (MSE): {mse}')
print(f'R-squared (R2) score: {r2}')

Mean squared error (MSE): 0.4763582491152158
R-squared (R2) score: 0.40862299182015016


In [13]:
# Assume user_input is a dictionary or some data structure containing user-supplied input
# Example user input:
user_input = {
    'volatile acidity': 0.3,
    'citric acid': 0.2,
    'residual sugar': 2.5,
    'chlorides': 0.045,
    'free sulfur dioxide': 25,
    'total sulfur dioxide': 100,
    'density': 0.995,
    'pH': 3.4,
    'sulphates': 0.6,
    'alcohol': 11.0,
    'Type_Red Wine': 0,  # For one-hot encoded 'Type' column
    'Type_White Wine': 1,
}

In [14]:
# Keep track of feature names after one-hot encoding
feature_names = df.drop(columns=['quality']).columns

# Create a DataFrame for user input with the same feature names
user_input_df = pd.DataFrame([user_input], columns=feature_names)

# Standardize user input using the same scaler as used during training
user_input_scaled = scaler.transform(user_input_df.values)

# Make predictions
predicted_quality = model.predict(user_input_scaled)

# Print the predicted quality
print(f"Predicted Quality: {predicted_quality} {predicted_quality[0][0]}")


1/1 [==============================] - 0s 28ms/step
Predicted Quality: [[5.71119]] 5.711190223693848


c:\Users\lefte\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
